In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, BatchNormalization, Dropout, Activation, MaxPooling2D, Flatten
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.regularizers import l1
from keras.callbacks import ReduceLROnPlateau
import pandas as pd
import time

In [3]:
def read_data(data_dir, is_train):
    data = pd.read_csv(data_dir)
    if is_train:
        data    = data.sample(frac=1).reset_index(drop=True)
        labels  = data["label"]
        labels  = to_categorical(labels)
        data    = data.drop("label", axis=1)
        data    /= 255
        return data, labels
    else:
        data /= 255
        return data

In [4]:
def read_data_as_image(data_dir, is_train):
    data = pd.read_csv(data_dir)
    if is_train:
        data = data.sample(frac=1).reset_index(drop=True)
        labels = data["label"]
        labels = to_categorical(labels)
        data = data.drop("label", axis=1)
        data /= 255
        image_data = data.values
        image_data = image_data.reshape((-1, 28, 28, 1))
        return image_data, labels
    else:
        data /= 255
        image_data = data.values
        image_data = image_data.reshape((-1, 28, 28, 1))
        return image_data

In [5]:
def build_model(layer_size, input_shape, regularizer, drop_size, num_of_classes, lr):
    model = Sequential()

    model.add(Dense(layer_size, activity_regularizer=regularizer, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(drop_size))

    model.add(Dense(layer_size, activity_regularizer=regularizer))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(drop_size))

    model.add(Dense(num_of_classes))
    model.add(Activation("softmax"))

    optimizer = optimizers.Adam(lr=lr)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [6]:
def build_cnn_model(layer_size, input_shape, regularizer, drop_size, num_of_classes, lr):
    model = Sequential()

    model.add(Conv2D(layer_size, (3, 3), use_bias=False, input_shape=input_shape,
                     activity_regularizer=regularizer))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(drop_size))

    model.add(Conv2D(layer_size, (3, 3), use_bias=False, activity_regularizer=regularizer))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(drop_size))

    model.add(Conv2D(layer_size, (3, 3), use_bias=False, activity_regularizer=regularizer))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(drop_size))

    model.add(Flatten())
    model.add(Dense(units=int(layer_size/8), use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(drop_size))

    model.add(Dense(units=num_of_classes, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))

    optimizer = optimizers.Adam(lr=lr)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [7]:
train_data_dir  = "/kaggle/input/digit-recognizer/train.csv"
test_data_dir   = "/kaggle/input/digit-recognizer/test.csv"

train_data, train_labels = read_data_as_image(train_data_dir, True)
test_data = read_data_as_image(test_data_dir, False)

In [8]:
layer_size      = 2048
# input_shape     = [train_data.shape[1]]
input_shape     = (28, 28, 1)
regularizer     = l1(0.001)
drop_size       = 0.2
num_of_classes  = 10
lr              = 1e-5

In [9]:
model = build_cnn_model(layer_size, input_shape, regularizer, drop_size, num_of_classes, lr)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 2048)      18432     
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 2048)      8192      
_________________________________________________________________
activation (Activation)      (None, 26, 26, 2048)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 2048)      0         
_________________________________________________________________
dropout (Dropout)            (None, 13, 13, 2048)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 2048)      37748736  
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 11, 2048)      8

In [10]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=1e-8)

In [11]:
nb_epochs   = 75
val_split   = 0.2
batch_size  = 32

In [12]:
model.fit(train_data, train_labels, epochs=nb_epochs, validation_split=val_split,
          batch_size=batch_size, verbose=1, callbacks=[reduce_lr])

model.save("model" + str(time.time()) + ".h5")

Epoch 1/75
1050/1050 [==============================] - 140s 133ms/step - loss: 36.8164 - accuracy: 0.6878 - val_loss: 13.8385 - val_accuracy: 0.4107
Epoch 2/75
1050/1050 [==============================] - 139s 133ms/step - loss: 16.6524 - accuracy: 0.8195 - val_loss: 7.1001 - val_accuracy: 0.4193
Epoch 3/75
1050/1050 [==============================] - 139s 133ms/step - loss: 6.7684 - accuracy: 0.8525 - val_loss: 3.1717 - val_accuracy: 0.9110
Epoch 4/75
1050/1050 [==============================] - 139s 133ms/step - loss: 3.1361 - accuracy: 0.8721 - val_loss: 2.0389 - val_accuracy: 0.9406
Epoch 5/75
1050/1050 [==============================] - 140s 133ms/step - loss: 2.0624 - accuracy: 0.8972 - val_loss: 1.5057 - val_accuracy: 0.9456
Epoch 6/75
1050/1050 [==============================] - 140s 133ms/step - loss: 1.5537 - accuracy: 0.9146 - val_loss: 1.2597 - val_accuracy: 0.9533
Epoch 7/75
1050/1050 [==============================] - 139s 133ms/step - loss: 1.2377 - accuracy: 0.9263 - v

In [13]:
pred    = model.predict(test_data)
preds   = pred.argmax(axis=-1)

results = pd.DataFrame(preds, columns=["Label"])
results.index += 1
results.to_csv("submission" + str(time.time()) + ".csv", index_label="ImageId")